In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json
! kaggle competitions download riiid-test-answer-prediction
! unzip riiid-test-answer-prediction.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% 1.29G/1.29G [00:07<00:00, 190MB/s]
100% 1.29G/1.29G [00:07<00:00, 177MB/s]
Archive:  riiid-test-answer-prediction.zip
  inflating: example_sample_submission.csv  
  inflating: example_test.csv        
  inflating: lectures.csv            
  inflating: questions.csv           
  inflating: riiideducation/__init__.py  
  inflating: riiideducation/competition.cpython-37m-x86_64-linux-gnu.so  
  inflating: train.csv               


In [2]:
import numpy as np
import pandas as pd

In [3]:
# В связи с ограничениями оперативной памяти и вычислительной мощности используется уменьшенная наполовину версия датасета
# Полный датасет имеет форму (101230332, 10)
train_df = pd.read_csv(
    '/content/train.csv',
    nrows=50_000_000,
    dtype={
        'user_id': np.int32,
        'content_id': np.int16,
        'content_type_id': np.int8,
        'task_container_id': np.int16,
        'user_answer': np.int8,
        'answered_correctly': np.int8,
        'prior_question_elapsed_time': np.float32
    }
)

questions_df = pd.read_csv('/content/questions.csv')

lectures_df = pd.read_csv('/content/lectures.csv')

In [4]:
# Первые пять строк таблицы 'train'
train_df.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [5]:
# Детальная информация о столбцах таблицы 'train'
# Поскольку датасет большой, количество непустых значений метод info() не выводит
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000000 entries, 0 to 49999999
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int32  
 3   content_id                      int16  
 4   content_type_id                 int8   
 5   task_container_id               int16  
 6   user_answer                     int8   
 7   answered_correctly              int8   
 8   prior_question_elapsed_time     float32
 9   prior_question_had_explanation  object 
dtypes: float32(1), int16(2), int32(1), int64(2), int8(3), object(1)
memory usage: 1.8+ GB


In [7]:
# Столбец 'prior_question_elapsed_time' содержит 1_161_187 пустых значений NaN
# Столбец 'prior_question_had_explanation' имеет 192_960 пустых значений NaN
# Пустые значения NaN можно заменить, например, на среднее значение признаков в данной категории, либо,
# если датасет большой, а пустых значений немного, можно просто ими пренебречь
# Однако в описании к датасету указывается на то, что признак 'prior_question_elapsed_time' имеет значение null
# для первого набора вопросов пользователя или лекции
for i, column in enumerate(train_df.columns):
    print(f'{i} {column}: {train_df.shape[0] - train_df[column].isna().sum()} non-null')
    if train_df[column].isna().sum() > 0:
        print(f'\t{column}: {train_df[column].isna().sum()} is NaN')

0 row_id: 50000000 non-null
1 timestamp: 50000000 non-null
2 user_id: 50000000 non-null
3 content_id: 50000000 non-null
4 content_type_id: 50000000 non-null
5 task_container_id: 50000000 non-null
6 user_answer: 50000000 non-null
7 answered_correctly: 50000000 non-null
8 prior_question_elapsed_time: 48838813 non-null
	prior_question_elapsed_time: 1161187 is NaN
9 prior_question_had_explanation: 49807040 non-null
	prior_question_had_explanation: 192960 is NaN


In [8]:
# Описательная статистика таблицы 'train'
# Можно обратить внимание на столбец 'prior_question_elapsed_time', среднее время в миллисекундах,
# которое потребовалось пользователю, для ответа на каждый вопрос
# Среднее время ответа находится в пределах 25 секунд, а максимальное - 5 минут, разброс значений достаточно большой
train_df.describe()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time
count,5.000000e+07,5.000000e+07,5.000000e+07,5.000000e+07,5.000000e+07,5.000000e+07,5.000000e+07,5.000000e+07,4.883881e+07
mean,2.500000e+07,7.765412e+09,5.335441e+08,5.215880e+03,1.936454e-02,9.169790e+02,1.375844e+00,6.252737e-01,2.542749e+04
std,1.443376e+07,1.166085e+10,3.052798e+08,3.870612e+03,1.378026e-01,1.382089e+03,1.193027e+00,5.225281e-01,1.994180e+04
min,0.000000e+00,0.000000e+00,1.150000e+02,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00
25%,1.250000e+07,5.261356e+08,2.713496e+08,2.061000e+03,0.000000e+00,1.050000e+02,0.000000e+00,0.000000e+00,1.600000e+04
50%,2.500000e+07,2.694542e+09,5.344314e+08,5.013000e+03,0.000000e+00,3.840000e+02,1.000000e+00,1.000000e+00,2.100000e+04
75%,3.750000e+07,1.008010e+10,8.011038e+08,7.431000e+03,0.000000e+00,1.105000e+03,3.000000e+00,1.000000e+00,2.966600e+04
max,5.000000e+07,8.742577e+10,1.059622e+09,3.273600e+04,1.000000e+00,9.999000e+03,3.000000e+00,1.000000e+00,3.000000e+05


In [9]:
# Значения столбца 'prior_question_elapsed_time' для простоты восприятия преобразуем в секунды
train_df['prior_question_elapsed_time'] = train_df['prior_question_elapsed_time'] / 1_000

In [10]:
# Описательная статистика по столбцу 'prior_question_had_explanation'
# (видел ли пользователь объяснение и правильный ответ после ответа на предыдущий вопрос)
# В большинстве случаев пользователь видел объяснение и правильный ответ после ответа на вопрос
train_df.describe(include='object')

,prior_question_had_explanation
count,49807040
unique,2
top,True
freq,44338145


In [11]:
# Столбец 'row_id' является индексом и для анализа данных нам не понадобится
train_df.drop('row_id', axis=1, inplace=True)

In [12]:
# В целях оптимизации хранения данных и работы с ними столбцы 'content_type_id', 'user_answer', 'answered_correctly', 'prior_question_had_explanation'
# можно преобразовать к категориальному типу данных
# train_df['content_type_id'] = train_df['content_type_id'].astype('category')
# train_df['user_answer'] = train_df['user_answer'].astype('category')
# train_df['answered_correctly'] = train_df['answered_correctly'].astype('category')
# train_df['prior_question_had_explanation'] = train_df['prior_question_had_explanation'].astype('category')

In [13]:
# Количество уникальных пользователей образовательной платформы
users = train_df['user_id'].nunique()
users

193520

In [14]:
# Количество просмотренных пользователями лекций
lectures_count = train_df['content_type_id'].value_counts()[1]
lectures_count

968227

In [15]:
# Количество заданных пользователям вопросов
questions_count = train_df['content_type_id'].value_counts()[0]
questions_count

49031773

In [16]:
# Среднее количество вопросов, приходящихся на одну лекцию
round(questions_count / lectures_count, 2)

50.64

In [17]:
# Среднее количество просмотренных пользователем лекций
round(lectures_count / users, 2)

5.0

In [18]:
# Среднее количество заданных пользователю вопросов
round(questions_count / users, 2)

253.37

In [19]:
# Процент данных правильных ответов
correct_answers = train_df[(train_df['answered_correctly'] == 0) | 
                           (train_df['answered_correctly'] == 1)]['answered_correctly'].value_counts(normalize=True)
# 1    0.657368
# 0    0.342632
# Name: answered_correctly, dtype: float64
round(correct_answers[1] * 100, 2)

65.74

In [20]:
# Относительные значения данных пользователем правильных/неправильных ответов,
# когда он ВИДЕЛ объяснение и правильный ответ на предыдущий вопрос
# (1 - правильный ответ, 0 - неправильный ответ)
train_df[(train_df['prior_question_had_explanation'] == True) & 
         (train_df['answered_correctly'] == 0) | 
         (train_df['answered_correctly'] == 1)]['answered_correctly'].value_counts(normalize=True)

1    0.689928
0    0.310072
Name: answered_correctly, dtype: float64

In [21]:
# Относительные значения данных пользователем правильных/неправильных ответов,
# когда он НЕ ВИДЕЛ объяснение и правильный ответ на предыдущий вопрос
# В случае, когда пользователь НЕ ВИДЕЛ объяснение на предыдущий вопрос,
# он отвечал на последующий вопрос правильно в 93 % случаев
train_df[(train_df['prior_question_had_explanation'] == False) & 
         (train_df['answered_correctly'] == 0) | 
         (train_df['answered_correctly'] == 1)]['answered_correctly'].value_counts(normalize=True)

1    0.934693
0    0.065307
Name: answered_correctly, dtype: float64

In [22]:
# Однако к этому выводу следует относиться с осторожностью,
# поскольку выборка распределена неравномерно с преобладанием значений ВИДЕЛ
train_df['prior_question_had_explanation'].value_counts(normalize=True)

True     0.890198
False    0.109802
Name: prior_question_had_explanation, dtype: float64

In [23]:
# Если пользователь думал над вопросом по времени больше среднего значения,
# верность данного ответа на вопрос увеличивалась до 84 %
avg_time = train_df['prior_question_elapsed_time'].mean()
print('Среднее время ответа на вопрос:', avg_time)
train_df[(train_df['prior_question_elapsed_time'] > avg_time) &
         (train_df['answered_correctly'] == 0) | 
         (train_df['answered_correctly'] == 1)]['answered_correctly'].value_counts(normalize=True)

Среднее время ответа на вопрос: 25.427565


1    0.844181
0    0.155819
Name: answered_correctly, dtype: float64

In [24]:
# Общее количество времени, потраченного пользователями на ответы на вопросы
time_seconds = train_df['prior_question_elapsed_time'].sum()
print('В секундах:', round(time_seconds, 2))
print('В минутах:', round(time_seconds / 60, 2))
print('В часах:', round(time_seconds / 3_600, 2))
print('В сутках:', round(time_seconds / 86_400, 2))
print('В неделях:', round(time_seconds / 604_800, 2))
print('В месяцах:', round(time_seconds / 2_592_000, 2))
print('В годах:', round(time_seconds / 31_536_000, 2))

В секундах: 1241852000.0
В минутах: 20697533.87
В часах: 344958.9
В сутках: 14373.29
В неделях: 2053.33
В месяцах: 479.11
В годах: 39.38


In [25]:
# Вопросы, на которые пользователи потратили больше всего времени в секундах (первые пять в порядке убывания)
pd.pivot_table(
    train_df[train_df['content_type_id'] == 0],
    values='prior_question_elapsed_time',
    index='content_id',
    aggfunc=np.sum
).sort_values('prior_question_elapsed_time', ascending=False).head()

,prior_question_elapsed_time
content_id,
4120,2673996.75
7216,2532778.50
7217,2532778.50
7218,2532778.50
7219,2532773.25


In [26]:
# Самые популярные лекции среди пользователей платформы по количеству просмотров (первые пять в порядке убывания)
train_df[train_df['content_type_id'] == 1]['content_id'].value_counts().head()

3153     18202
21411    13431
10540    11942
29695    11896
15888    11095
Name: content_id, dtype: int64

In [27]:
# Самые популярные вопросы среди пользователей платформы по количеству данных ответов (первые пять в порядке убывания)
train_df[train_df['content_type_id'] == 0]['content_id'].value_counts().head()

6116    104702
6173     99083
4120     97591
175      95984
7876     93240
Name: content_id, dtype: int64

In [28]:
# Самые сложные вопросы на платформе (первые пять в порядке убывания)
# Самый популярный вопрос является одновременно и самым сложным вопросом на платформе
train_df[(train_df['content_type_id'] == 0) & (train_df['answered_correctly'] == 0)]['content_id'].value_counts().head()

6116    75289
2063    72355
4120    70717
6173    69882
2946    64728
Name: content_id, dtype: int64

In [29]:
# Около 72 % ответов на самый сложный вопрос являются неверными
train_df[(train_df['content_type_id'] == 0) & (train_df['content_id'] == 6116)]['answered_correctly'].value_counts(normalize=True)

0    0.719079
1    0.280921
Name: answered_correctly, dtype: float64

In [30]:
# Среднее время, потраченное на ответ на самый сложный вопрос в секундах
train_df[train_df['content_id'] == 6116]['prior_question_elapsed_time'].mean()

19.993883

In [31]:
# Максимальное время, потраченное на ответ на самый сложный вопрос в секундах
train_df[train_df['content_id'] == 6116]['prior_question_elapsed_time'].max()

300.0

In [32]:
# Самые лёгкие вопросы на платформе (первые пять в порядке убывания)
train_df[(train_df['content_type_id'] == 0) & (train_df['answered_correctly'] == 1)]['content_id'].value_counts().head()

7900    73175
294     56134
2064    55208
2065    54844
1278    54466
Name: content_id, dtype: int64

In [33]:
# Более 82 % ответов на самый лёгкий вопрос являются верными 
train_df[(train_df['content_type_id'] == 0) & (train_df['content_id'] == 7900)]['answered_correctly'].value_counts(normalize=True)

1    0.824944
0    0.175056
Name: answered_correctly, dtype: float64

In [34]:
# Среднее время, потраченное на ответ на самый лёгкий вопрос в секундах
# Это время превышает среднее время, потраченное на самый сложный вопрос (19.993883)
train_df[train_df['content_id'] == 7900]['prior_question_elapsed_time'].mean()

21.555458

In [35]:
# Первые пять строк таблицы 'lectures'
lectures_df.head()

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [36]:
# Детальная информация о столбцах таблицы 'lectures'
lectures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int64 
 1   tag         418 non-null    int64 
 2   part        418 non-null    int64 
 3   type_of     418 non-null    object
dtypes: int64(3), object(1)
memory usage: 13.2+ KB


In [37]:
# Описательная статистика таблицы 'lectures'
lectures_df.describe()

,lecture_id,tag,part
count,418.000000,418.000000,418.000000
mean,16983.401914,94.480861,4.267943
std,9426.164660,53.586487,1.872424
min,89.000000,0.000000,1.000000
25%,9026.250000,50.250000,2.000000
50%,17161.500000,94.500000,5.000000
75%,24906.250000,140.000000,6.000000
max,32736.000000,187.000000,7.000000


In [38]:
# Описательная статистика по столбцу 'type_of' (основные цели лекций)
# Количество уникальных значений 4
# Самым частым значением является 'concept' с частотой появления 222
lectures_df.describe(include='object')

,type_of
count,418
unique,4
top,concept
freq,222


In [39]:
# Другие значения и частота использования основных целей лекций
lectures_df['type_of'].value_counts()

concept             222
solving question    186
intention             7
starter               3
Name: type_of, dtype: int64

In [40]:
# Количество лекций на платформе
lectures_df['lecture_id'].nunique()

418

In [41]:
# Первые пять строк таблицы 'questions'
questions_df.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [42]:
# Детальная информация о столбцах таблицы 'questions'
# В столбце 'tags' есть одно пустое значение NaN
questions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int64 
 1   bundle_id       13523 non-null  int64 
 2   correct_answer  13523 non-null  int64 
 3   part            13523 non-null  int64 
 4   tags            13522 non-null  object
dtypes: int64(4), object(1)
memory usage: 528.4+ KB


In [43]:
# Описательная статистика таблицы 'questions'
questions_df.describe()

,question_id,bundle_id,correct_answer,part
count,13523.00000,13523.000000,13523.000000,13523.000000
mean,6761.00000,6760.510907,1.455298,4.264956
std,3903.89818,3903.857783,1.149707,1.652553
min,0.00000,0.000000,0.000000,1.000000
25%,3380.50000,3379.500000,0.000000,3.000000
50%,6761.00000,6761.000000,1.000000,5.000000
75%,10141.50000,10140.000000,3.000000,5.000000
max,13522.00000,13522.000000,3.000000,7.000000


In [44]:
# Описательная статистика по столбцу 'tags' (коды тегов для вопросов)
# Количество уникальных значений 1519
# Самым частым значением является 8 с частотой появления 738
questions_df.describe(include='object')

,tags
count,13522
unique,1519
top,8
freq,738


In [45]:
# Другие значения и частота использования тегов для вопросов (первые пять строк по убыванию)
questions_df['tags'].value_counts().head()

8     738
73    617
53    523
1     413
96    373
Name: tags, dtype: int64

In [46]:
# Количество вопросов на платфоре
questions_df['question_id'].nunique()

13523

In [47]:
# Количество вариантов ответов на вопрос
questions_df['correct_answer'].nunique()

4

In [48]:
# В большинстве вопросов правильный ответ находится под номером 0
questions_df['correct_answer'].value_counts(normalize=True)

0    0.274791
3    0.262072
1    0.257191
2    0.205945
Name: correct_answer, dtype: float64

In [49]:
# Количество составных частей теста на образовательной платформе
lectures_df['part'].nunique() == questions_df['part'].nunique()
# True
lectures_df['part'].nunique()

7

In [50]:
# Количество лекций в каждой части теста
lectures_df.groupby(['part'])['lecture_id'].count()

part
1     54
2     56
3     19
4     31
5    143
6     83
7     32
Name: lecture_id, dtype: int64

In [51]:
# Количество вопросов в каждой части теста
questions_df.groupby(['part'])['question_id'].count()

part
1     992
2    1647
3    1562
4    1439
5    5511
6    1212
7    1160
Name: question_id, dtype: int64